In [1]:
# pip install c3d

In [2]:
#import c3d, csv, os
#older c3d library doesn't support EVENT parameters
from ezc3d import c3d as myc3d
import csv, os

In [3]:
counter = 0
filepaths = []
for root, dirs, files in os.walk('data'):
    if len(files) > 1:
        for file in files:
            if 'ST' not in file:
                counter += 1    
                filepath = root + "\\" + file
                filepaths.append(filepath)
filepaths.sort()
print("Number of C3D files found:", counter)

In [4]:
%%capture cap --no-stderr
# we only read the labels once from one file since they are the same in all files
file = "data/2014001/2014001_C1_01.c3d"
%run -i c3d-metadata $file

In [5]:
with open('output.txt', 'w') as f:
    f.write(cap.stdout)

with open('footsteps.csv', mode='w') as footsteps:
    writer = csv.writer(footsteps)
    # frame data starts here
    count = 0
    for path in filepaths:
        handle = open(path, 'rb')
        reader = c3d.Reader(handle)
        ID = handle.name.split('\\')[2].split('_')[0]
        speed = handle.name.split('_')[1][1]
        trial = handle.name.split('_')[2].split('.')[0][1]
        counter = 0
        row = ID + " " + speed + " " + trial + " "
        item = myc3d(path)
        ctx = item['parameters']['EVENT']['CONTEXTS']['value']
        labels = item['parameters']['EVENT']['LABELS']['value']
        times = item['parameters']['EVENT']['TIMES']['value'][1]
        for i in range(0, 7):
            row += str(ctx[i]).lower() +"_"+ str(labels[i]).lower().replace(" ", "_") + ' ' + str(times[i]) +' '
        writer.writerow(row.split(' '))

In [6]:
#Had to do some manual modification too
